In [61]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline
import warnings

from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
mnist = datasets.load_digits()

In [62]:
X = mnist['data']
y = mnist['target']

# Data Splitting

In [63]:
train_size = 0.8

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.astype(np.float64))
X_train

array([[ 0.        ,  1.89304218,  1.46344076, ..., -1.14020226,
        -0.50841639, -0.19912413],
       [ 0.        ,  0.78180786,  0.18878815, ...,  0.20791717,
        -0.50841639, -0.19912413],
       [ 0.        , -0.32942646, -0.66098027, ...,  0.03940224,
        -0.50841639, -0.19912413],
       ...,
       [ 0.        , -0.32942646, -0.66098027, ...,  1.38752166,
         0.69917758, -0.19912413],
       [ 0.        , -0.32942646, -0.44853816, ..., -0.63465747,
        -0.50841639, -0.19912413],
       [ 0.        , -0.32942646,  1.46344076, ...,  0.37643209,
        -0.50841639, -0.19912413]])

# Grid Search - Precision Scoring

In [64]:
svc_prec = svm.SVC(kernel='linear', gamma='auto', C=2, random_state=42)

In [65]:
# Remove 'precomputed' from your parameter space.
# kernel='precomputed'can only be used when passing a (n_samples, n_samples) data matrix that represents pairwise similarities for the samples instead of the traditional (n_samples, n_features) rectangular data matrix.

param_grid = [
    {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
        'gamma': [1e-3, 1e-4],
        'C': [1, 10, 100, 1000, 5000]
    }
]

In [66]:
grid_search_pres = GridSearchCV(svc_prec, param_grid, cv=5, scoring='precision_weighted')
grid_search_pres

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='linear', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000, 5000],
                          'gamma': [0.001, 0.0001],
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision_weighted', verbose=0)

In [67]:
grid_search_pres.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='linear', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000, 5000],
                          'gamma': [0.001, 0.0001],
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='precision_weighted', verbose=0)

In [68]:
grid_search_pres.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [69]:
grid_search_pres.best_estimator_

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [70]:
sgd_cvres = grid_search_pres.cv_results_

for param, score in zip(sgd_cvres['params'], sgd_cvres['mean_test_score']):
    print(score, param)


0.9728983673744265 {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
0.010759532473131224 {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}
0.94454287750469 {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.9257998295138287 {'C': 1, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.9728983673744265 {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
0.010759532473131224 {'C': 1, 'gamma': 0.0001, 'kernel': 'poly'}
0.6009885195743979 {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.04626518146334604 {'C': 1, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.9728983673744265 {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
0.11229070466312084 {'C': 10, 'gamma': 0.001, 'kernel': 'poly'}
0.9804618803932585 {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.977861433202078 {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.9728983673744265 {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.010759532473131224 {'C': 10, 'gamma': 0.0001, 'kernel': 'poly'}
0.945451233833934 {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.9250891365623564 {'C': 10, 'gamma': 0

# Grid Search - Recall Scoring

In [71]:
svc_recall = svm.SVC(kernel='linear', gamma='auto', C=2, random_state=42)

In [72]:
grid_search_recall = GridSearchCV(svc_recall, param_grid, cv=5, scoring='recall_macro')

In [73]:
grid_search_recall.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto', kernel='linear', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000, 5000],
                          'gamma': [0.001, 0.0001],
                          'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='recall_macro', verbose=0)

In [74]:
grid_search_recall.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [75]:
grid_search_recall.best_estimator_

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [76]:
cvres = grid_search_recall.cv_results_

for param, score in zip(cvres['params'], cvres['mean_test_score']):
    print(score, param)

0.9723610201725528 {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
0.1 {'C': 1, 'gamma': 0.001, 'kernel': 'poly'}
0.9415908521587261 {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.9208675494023321 {'C': 1, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.9723610201725528 {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
0.1 {'C': 1, 'gamma': 0.0001, 'kernel': 'poly'}
0.37121952320390156 {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.11993265023716397 {'C': 1, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.9723610201725528 {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
0.1054625296954863 {'C': 10, 'gamma': 0.001, 'kernel': 'poly'}
0.9797499992699522 {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.9771144690768909 {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.9723610201725528 {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.1 {'C': 10, 'gamma': 0.0001, 'kernel': 'poly'}
0.9422854837175859 {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.9201716551782542 {'C': 10, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.9723610201725528 

# Final Testing

In [77]:
shuffle_index = np.random.permutation(len(X_test))
X_test, y_test = X_test[shuffle_index], y_test[shuffle_index]

scaler = StandardScaler()
X_test = scaler.fit_transform(X_test.astype(np.float64))
X_test

array([[ 0.        , -0.35691531, -0.32091645, ..., -1.17359866,
        -0.49529291, -0.18670359],
       [ 0.        , -0.35691531,  0.2892768 , ...,  0.38991271,
        -0.49529291, -0.18670359],
       [ 0.        , -0.35691531,  1.10286781, ..., -0.82615169,
        -0.49529291, -0.18670359],
       ...,
       [ 0.        , -0.35691531, -0.1175187 , ..., -1.17359866,
        -0.49529291, -0.18670359],
       [ 0.        ,  2.85532244,  0.89947006, ..., -0.82615169,
        -0.49529291, -0.18670359],
       [ 0.        , -0.35691531, -1.13450746, ..., -1.17359866,
        -0.49529291, -0.18670359]])

In [78]:
final_model = grid_search_pres.best_estimator_

In [79]:
final_predictions = final_model.predict(X_test)

In [81]:
sum(final_predictions == y_test) / len(X_test)

0.9888888888888889